# Classification with trees and forests

## Decision Trees


### Overview

Decision tree (DT) is a *non-parametric* supervised method used for both regression and classification. As the name suggests, DTs uses a tree-like model where each internal node rappresent a test on one (or more) attributes of our dataset, and each leaf node rappresent a class label; It follows that the branches rappresent the outcome of our test. The path from root to leaf are the classifications rules.

![A simple rappresentation of a decision tree](imgs/simpleDT.jpg#center)

\
Decision Trees have many advantages such as : 
* Simple to understand and even visualize
* It is a *white box* model; Non parametric approach that is no assumption on the shape/distribution of the data
* Can work with both numerical and categorical values
* Easy train and fast performances



On the other hand decision trees can be very sensitive to small change in the data that can result in major change in the structure of the tree; Another problem with this approach is the danger of overfitting if not taken enough precautions. 



**So the question is how can we can construct a decision tree?**

### The building process: Reducing impurity

The algorithm to build a decision is a 'greedy' algorithm that at each node try to find the variable that **best split** the set of items in each step. So the question becomes what is considered the best split ? 

Two main metrics are used nowdays : 
* Gini impurity 
* Information Gain / Entropy impurity

In this brief project we decided to use the latter.

In general we define the information gain as:   
$$IG(T, \alpha ) = H(T) - H(T| \alpha )$$
where $$H(X) = - \sum^n_{i=1} P(x_i)logP(x_i)$$
is the entropy. 

So when building the decision tree we are trying to reduce the conditional entropy that is equivalent to maximise the information gain on each split. In simple terms we are trying to learn $\alpha$ such that our uncertainty about our observations is minimized. 

So the recursive process will be to begin with the whole dataset in the root node. We then calculate which is the best split among all features and all possible values for that feature; This will leave us with a threshold and feature on where to split. We assign to the left child of the root all observations where it's value is less or equal than the threshold and the rest to the right child. The process in then recursively repeated untill we either have all obs belonging to one class or where there are not enough observations to further split. In both cases those nodes will be the leaf nodes.


For a more Detailed implentation refer to the **DecisionTree\tree.py** python class.

## A little showcase

In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
dF = pd.read_csv('data//csvs//dataframeV1.csv', index_col=0)

# Drop not usefull cols
dF = dF.drop(['id', 'uri'], axis = 1)

# Turn genres names into cat 
dF.label = pd.Categorical(dF.label)
dF['Y'] = dF.label.cat.codes
dF = dF.drop(['label'], axis = 1)

# Shuffling the dataset
dF = dF.sample(frac=1).reset_index(drop=True)

dF.head()

In [ ]:
from DecisionTree.tree import Tree

# Creating the decision Tree
dT = Tree()

In [ ]:
# Prepare the dataset for the decision tree

y = dF.Y 
X = dF.drop(["Y"], axis=1)

# Split in train and validation
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=43, train_size=.75)

In [ ]:
# Fit the tree on the train set 
dT.fit(x_train.to_numpy(), y_train.to_numpy())

In [ ]:
# Build the prediction over the validation set
preds = dT.predict(x_test.to_numpy())

In [ ]:
# Calculate the overall accuracy.
sum(preds == y_test) / len(preds)

# random forest


We now try to address one of the main problems when using DTs, that is overfitting. In fact is common for such models to learn on the noise and small variations in the data, if hyper-paramethers are not carefully tuned. 

A solution to this is to use Random Forests (RF). RF is an **ensamble learning** method that operates by constructing multiple decision trees. Many decision trees are constructed applying the general technique of bootstrapping.

More formally if we have a training set $X = x_1, ..., x_n$ and the labels associated $Y = y_1, ...,y_n$ we **Bag** (Sample with replacment) from $\{X,Y\}$ $ B $ times and train a decision tree on this sample. We then in case of classification take the majority of votes from all the $B$ predictions as the final output of the RF model.

Bootstrapping so help us solve the issue of overfitting. A step further in this direction would be modify the DT algorithm slightly, by looking at each step to only a portion of the features to find the best possible split. This not only improve the overall model accuracy and speed but also provide a further guard against overfitting. 

In [ ]:
from RandomForest.randomForest import Forest

So the first question is how many trees should build ? As there is no universal answer the best way to find out this hyperparameter is to use some model selection techniques. 

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
parameters = {
    'max_trees' : [x for x in range(24,27)],
    'max_detph' : [4,5,6,7]
}

K = 10


res_acc = {}
res_time = {}

# Set to true if need to rerun the cross validation (note: it takes some time)
doCV = False

if doCV : 
    for b in parameters['max_trees']: 
        res_acc['T' + str(b)] = list()
        res_time['T' + str(b)] = list()
        for d in parameters['max_detph']:
            print(f"Fitting Forest ID : T{b}D{d}", end = '\r')
            score = cross_validate(Forest(max_trees=b, max_depth=6), X, y, scoring='accuracy', cv = K, n_jobs=-1 )
            res_acc['T' + str(b)].append(sum(score['test_score']) / K)
            res_time['T' + str(b)].append(sum(score['score_time']) / K)



In [ ]:
if doCV : 
    acc_df = pd.DataFrame.from_dict(res_acc, orient='columns')
    acc_df = pd.concat([pd.Series(parameters['max_detph'], name = 'Depth'),acc_df], axis = 1)
    acc_df = acc_df.set_index('Depth')

    time_df = pd.DataFrame.from_dict(res_time, orient='columns')
    time_df = pd.concat([pd.Series(parameters['max_detph'], name = 'Depth'),time_df], axis = 1)
    time_df = time_df.set_index('Depth')
else : 
    acc_df = pd.read_csv('cv_acc_res.csv')
    acc_df = acc_df.set_index('Depth')
    time_df = pd.read_csv('cv_time_res.csv')
    time_df = time_df.set_index('Depth')

In [ ]:
acc_df

In [ ]:
time_df

In [ ]:
import json

save = False


if save:
    with open('cv_acc.csv', 'w') as f: 
        acc_df.to_csv(f)
        f.close()
    with open('cv_time.csv', 'w') as f: 
        time_df.to_csv(f)
        f.close()



In [ ]:
f = plt.figure()

acc_df.plot(ax = f.gca(), use_index = True)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.title("Accuracy of CV models")
plt.xlabel("Depth")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
f = plt.figure()

time_df.plot(ax = f.gca(), use_index = True)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.title("Time to fit")
plt.xlabel("Depth")
plt.ylabel("Time(s)") 
plt.show()
plt.show()